<a href="https://colab.research.google.com/github/Muhammadsulton1/MIPT_Data_analys/blob/main/%D0%BF%D1%80%D0%BE%D0%B4%D1%83%D0%BA%D1%82%D0%BE%D0%B2%D1%8B%D0%B5_%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D0%BA%D0%B8_%D0%B8_%D0%B4%D0%BE%D0%B2%D0%B5%D1%80%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%B2%D0%B0%D0%BB%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')  # стиль для графиков
%matplotlib inline

Чтобы держать руку на пульсе бизнеса, нужно строить довольно много продуктовых метрик. Более того, нужно понимать насколько адекватно эти метрики построены и насколько сильно они могут колебаться. Мы в этой тетрадке рассмотрим два таких показателя: возвращаемость (retention) и среднюю выручку (revenue per user, RPU)

In [ ]:
visits = pd.read_csv('marketing_log.csv', sep='\t')

# изменим тип колонки с датой на время
visits['date'] = pd.to_datetime(visits["date"], format="%Y-%m-%d")

# отсортируем данные по дате
visits.sort_values('date', inplace=True)

# сбросим индексирование таблики
visits.reset_index(drop=True, inplace=True)

print(visits.shape)
visits.head()

1. Число пользователей
Сколько людей пользуются в день, неделю, месяц?

In [ ]:
day = visits.groupby('date').agg({'fullVisitorId': 'nunique'})
day.head()

In [ ]:
print(f"Среднее число посетителей в день: {day.fullVisitorId.mean()}")
day.sort_index().plot(figsize=(12,6));

2. Возвращаемость (retention)
Первый показатель, который обычно интересует бизнес это возвращаемость (retention):

retention_1 - вернулся ли игрок после первого дня с момента посещения магазина?
retention_7 - вернулся ли игрок после седьмого дня с момента посещения магазина?
Заведём новую колонку: дата первого появления юзера.

In [ ]:
first_visit = visits.groupby('fullVisitorId').agg({'date':'min'})
first_visit.columns = ['firstVisit']

visits = visits.join(first_visit, on='fullVisitorId')
visits.head()

In [ ]:
visits['daysFromFirstVisit'] = (visits.date - visits.firstVisit).apply(lambda w: w.days)

In [ ]:
visits['daysFromFirstVisit'].hist(bins=50);

Видно, что чаще всего первое посещение оказывается последним

In [ ]:
visits[visits['daysFromFirstVisit'] > 20]['daysFromFirstVisit'].hist(bins=50);

Найдём для каждого человека разность в днях между первым и последним посещениями:

In [ ]:
firsVisits = visits.groupby(['firstVisit', 'fullVisitorId']).agg({'daysFromFirstVisit': 'max'}).reset_index()
firsVisits.head()

In [ ]:
visits.shape

In [ ]:
firsVisits.shape

Как посчитать retention_7? Если daysFromFirstVisit оказывается больше  7 , значит человек вернулся более, чем через  7  дней после первого посещения. Такие люди нас и интересуют. Найдём их количество.

In [ ]:
retention = (
    firsVisits.groupby('firstVisit')['daysFromFirstVisit']
    .agg([ # больше 1 => вернулся хотябы через день
        ("success", lambda w: sum(w >= 7)),
        ("total", "count") # сколько всего людей зашли в эту дату
    ])
)

retention.head()

Построим график.

In [ ]:
retention['retention'] = retention['success']/retention['total']
retention['retention'][30:120].plot(figsize=(14,6));

Есть много точечных оценок доли людей, которая возвращается для каждой даты. Но этого мало, каждая точка строится по разному числу наблюдений и нам хотелось бы понимать насколько оценка точная. Возвращаемость - это доля. Значит можно построить для неё доверителльный интервал с помощью ЦПТ:

$$
\hat p \pm z_{1 - \frac{\alpha}{2}} \cdot \sqrt{\frac{\hat p \cdot (1 - \hat p)}{n}}
$$

In [ ]:
retention.head()

In [ ]:
alpha = 0.05

# стандартная ошибка
retention['se'] = np.sqrt(retention['retention'] * (1 - retention['retention']) / retention['total'])

# границы интервалов
q = stats.norm.ppf(1 - alpha/2)
retention['left'] = retention['retention'] - q * retention['se']
retention['right'] = retention['retention'] + q * retention['se']

retention.head()

In [ ]:
df = retention[30:120]

df['retention'].plot(figsize=(14,6))
plt.fill_between(df.index, df['left'], df['right'], facecolor='blue', alpha=0.2, interpolate=True)
plt.show()

# 3. Средний доход с пользователя (RPU)

Посмотрим как часто люди покупают.

In [ ]:
# Процент покупок
100*(1 - visits['transactionRevenue'].isnull().sum()/visits.shape[0])

Всего лишь в $1\%$ случаев. Это очень редко, но с этой правдой жизни ничего не поделаешь. Оставим только тех, кто правда что-то покупал.

In [ ]:
# будем строить RPU на месячной основе
visits['month'] = visits['date'].apply(lambda w: w.strftime('%Y-%m'))

purchases = visits.dropna(subset=['transactionRevenue'])
purchases.shape

Посмотрим на распределение стоимостей покупок.

In [ ]:
purchases['transactionRevenue'].hist(bins=50);

Видим, что есть выбросы. Удалим их из выборки. Для того, что применять ЦПТ, мы предполагаем, что ни одна случайная величина из выборки не выделяется на фоне остальных

In [ ]:
q99 = purchases['transactionRevenue'].quantile(0.99)
q99

In [ ]:
purchases = purchases[purchases['transactionRevenue'] < q99]
purchases['transactionRevenue'].hist(bins=50);

Посчитаем среднее, стандартное отклонение и число наблюдений для каждого месяца.

In [ ]:
datePurchases = (
    visits.groupby(['month'])['transactionRevenue']
    .agg([('rpu', 'mean'),
          ('count', 'count'),
          ('se', 'std')])
    .reset_index()
)

In [ ]:
datePurchases.head()

Наконец, воспользуемся формулой для строительства доверительного интервала для среднего:

$$
\hat \mu \pm z_{1 - \frac{\alpha}{2}} \cdot \sqrt{\frac{\hat{\sigma}^2}{n}}
$$

In [ ]:
alpha = 0.05

# границы интервалов
q = stats.norm.ppf(1 - alpha/2)
datePurchases['left'] = datePurchases['rpu'] - q * datePurchases['se']/np.sqrt(datePurchases['count'])
datePurchases['right'] = datePurchases['rpu'] + q * datePurchases['se']/np.sqrt(datePurchases['count'])

datePurchases.tail()

Изобразим динамику RPU выручки на графике:

In [ ]:
datePurchases['rpu'].plot(figsize=(14,6))
plt.fill_between(datePurchases['month'], datePurchases['left'], datePurchases['right'],
                 facecolor='blue', alpha=0.2, interpolate=True)
plt.show()